<a href="https://colab.research.google.com/github/trefftzc/cis677/blob/main/Intro_to_OpenMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An Introduction to OpenMP
Examples taken from the book "Programming your GPU with OpenMP" by Tom Deakin and Tomothy G. Mattson

In [ ]:
%%writefile hello.c
#include <stdio.h>
#include <omp.h>
int main()
{
  #pragma omp parallel
  {
    int id = omp_get_thread_num();
    printf("hello %d",id);
    printf(" world %d\n",id);
  }
}


Writing hello.c


Now, compile the file hello.c
If one is using the gcc compiler, the appropriate flag is
 -fopenmp

In [ ]:
!gcc hello.c -o hello -fopenmp

Now, execute the program:

In [ ]:
!./hello

hello 1 world 1
hello 0 world 0


The default number of threads in a COLAB environment is 2.
The command lscpu in linux allows us to learn more about the characteristics of the CPU running the system.

In [ ]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  2
  On-line CPU(s) list:   0,1
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:          6
    Model:               79
    Thread(s) per core:  2
    Core(s) per socket:  1
    Socket(s):           1
    Stepping:            0
    BogoMIPS:            4399.99
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clf
                         lush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_
                         good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fm
                         a cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hyp
                         ervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd i

## Calculating Pi
This mathematical expression produces the constant pi.

$$ \int_{0}^{1} \frac{4.0}{(1 + x^2)} dx = \pi $$

It can be approximated as the sum or rectangles

$$ \sum_{i = 0}^{N} F(x_i) Δx ≈ \pi $$

In [ ]:
%%writefile pi.c
//
// Sequential version
//
#include <stdio.h>
#include <omp.h>

static long num_steps = 1024*1024*1024;

int main()
{
  double x, pi, step, sum = 0.0;
  step = 1.0 / (double) num_steps;

  for(int i = 0;i < num_steps;i++) {
    x = (i+0.5) * step;
    sum += 4.0 / (1.0 + x*x);
  }
  pi = step * sum;
  printf("Pi = %lf , with %ld steps\n ",pi,num_steps);
}

Writing pi.c


In [ ]:
!gcc pi.c -o pi -O3

In [ ]:
!./pi

Pi = 3.141593 , with 1073741824 steps
 

Using the time command to find out how long does the execution of the program takes.


In [ ]:
!time ./pi

Pi = 3.141593 , with 1073741824 steps
 
real	0m2.553s
user	0m2.498s
sys	0m0.003s


In [ ]:
%%writefile pi_spmd.c
//
// SPMD version
//
#include <stdio.h>
#include <omp.h>

static long num_steps = 1024*1024*1024;

int main()
{
  int numthreads;
  double pi, step, full_sum = 0.0;
  step = 1.0 / (double) num_steps;

#pragma omp parallel
  {
    int id = omp_get_thread_num();
    double x, partial_sum = 0.0;
    #pragma omp single
      numthreads = omp_get_num_threads();

    for(int i = id;i < num_steps;i += numthreads) {
      x = (i+0.5) * step;
      partial_sum += 4.0 / (1.0 + x*x);
    }
    #pragma omp critical
      full_sum += partial_sum;
}
  pi = step * full_sum;
  printf("Pi = %lf , with %ld steps\n ",pi,num_steps);
}

Overwriting pi_spmd.c


The compilation:

In [ ]:
!gcc pi_spmd.c -o pi_spmd -O3 -fopenmp

The execution and its timing.

In [ ]:
!time ./pi_spmd

Pi = 3.141593 , with 1073741824 steps
 
real	0m1.924s
user	0m3.635s
sys	0m0.006s


## The meaning of the different omp directives

1.   pragma omp parallel: The block that follows will be converted into a thread. Every available thread will execute this block in parallel.
2.   pragma omp single: Only one of the threads will execute the line or block that follows this pragma
3.   pragma omp critical: Only one thread will execute the line below at a time to avoid concurrent access problems.

## The meaning of the different omp functions


1.   omp_get_thread_num() Every thread has an unique id. The range starts and 0 and goes all the way up to the number of available threads minus one.
2.   omp_get_num_threads() Find the total number of threads that will execute





## A second version with OpenMP

In [ ]:
%%writefile pi_parallel_for.c
//
// Parallel for version
//
#include <stdio.h>
#include <omp.h>

static long num_steps = 1024*1024*1024;

int main()
{
  int numthreads;
  double x,pi, step, sum = 0.0;
  step = 1.0 / (double) num_steps;

#pragma omp parallel
  {
    int id = omp_get_thread_num();
    double x, partial_sum = 0.0;
    #pragma omp single
      numthreads = omp_get_num_threads();

    #pragma omp for private(x) reduction(+:sum)
    for(int i = 0;i < num_steps;i++) {
      x = (i+0.5) * step;
      sum = sum + 4.0 / (1.0 + x*x);
    }
}
  pi = step * sum;
  printf("Pi = %lf , with %ld steps\n ",pi,num_steps);
}

Overwriting pi_parallel_for.c


The compilation:

In [ ]:
!gcc pi_parallel_for.c -o pi_parallel_for -O3 -fopenmp

The execution and the timing:

In [ ]:
!time ./pi_parallel_for

Pi = 3.141593 , with 1073741824 steps
 
real	0m2.089s
user	0m3.632s
sys	0m0.006s


## Using tasks to parallelize recursive code

In [ ]:
%%writefile pi_with_tasks.c
//
// Parallel recursive version with tasks
//
#include <omp.h>
#include <stdio.h>
static long num_steps = 1024*1024*1024;
#define MIN_BLK 1024*256

double pi_comp(int Nstart,int Nfinish,double step)
{
  double x, sum = 0.0,sum1,sum2;
  // Base case.
  // This is a relatively small sub-block
  // Don't use recursion.
  if (Nfinish -Nstart < MIN_BLK)
  {
    for (int i = Nstart;i < Nfinish;i++)
    {
      x = (i + 0.5) * step;
      sum = sum + 4.0 / (1.0 + x * x);
    }
  }
  else // Recursive case
  {
    int iblk = Nfinish - Nstart;
    #pragma omp task shared(sum1)
      sum1 = pi_comp(Nstart,Nfinish-iblk/2,step);
    #pragma omp task shared(sum2)
      sum2 = pi_comp(Nfinish - iblk/2, Nfinish, step);
    #pragma om taskwait
      sum = sum1 + sum2;
  }
  return sum;
}

int main()
{
  double step,pi,sum;
  step = 1.0 / (double) num_steps;
  #pragma omp parallel
    #pragma omp single
      sum = pi_comp(0,num_steps,step);
  pi = step * sum;
  printf("Pi = %lf , with %ld steps\n ",pi,num_steps);
}

Overwriting pi_with_tasks.c


The compilation:

In [ ]:
!gcc pi_with_tasks.c -o pi_with_tasks -O3 -fopenmp

The execution and timing:

In [ ]:
!time ./pi_with_tasks

Pi = 0.000000 , with 1073741824 steps
 
real	0m1.553s
user	0m2.897s
sys	0m0.004s


# Now programming GPU with OpenMP
Notice the two new directives:


1.   pragma omp target
2.   pragma omp loop

target instructs the compiler to offload the execution to the GPU.
loop instructs the compiler to parallelize the loop on the GPU.



In [ ]:
%%writefile vadd_target.c
#include <stdio.h>
#include <omp.h>
// #define N 10000000
#define N 10000
#define TOL  0.0000001
//
//  This is a simple program to add two vectors
//  and verify the results.
//
//  History: Written by Tim Mattson, November 2017
//
int main()
{

    float a[N], b[N], c[N], res[N];
    int err=0;

    double init_time, compute_time, test_time;
    init_time    = -omp_get_wtime();

   // fill the arrays
   #pragma omp parallel for
   for (int i=0; i<N; i++){
      a[i] = (float)i;
      b[i] = 2.0*(float)i;
      c[i] = 0.0;
      res[i] = i + 2*i;
   }

   init_time    +=  omp_get_wtime();
   compute_time  = -omp_get_wtime();

   // add two vectors
   #pragma omp target
   #pragma omp loop
   for (int i=0; i<N; i++){
      c[i] = a[i] + b[i];
   }

   compute_time +=  omp_get_wtime();
   test_time     = -omp_get_wtime();

   // test results
   #pragma omp parallel for reduction(+:err)
   for(int i=0;i<N;i++){
      float val = c[i] - res[i];
      val = val*val;
      if(val>TOL) err++;
   }

   test_time    +=  omp_get_wtime();

   printf(" vectors added with %d errors\n",err);

   printf("Init time:    %.3fs\n", init_time);
   printf("Compute time: %.3fs\n", compute_time);
   printf("Test time:    %.3fs\n", test_time);
   printf("Total time:   %.3fs\n", init_time + compute_time + test_time);
   return 0;
}

Writing vadd_target.c


Now the compilation:

In [ ]:
!gcc vadd_target.c -o vadd_target -fopenmp

And the execution:

In [ ]:
!export OMP_TARGET_OFFLOAD=mandatory
!./vadd_target

 vectors added with 0 errors
Init time:    0.008s
Compute time: 0.001s
Test time:    0.000s
Total time:   0.008s


And using the nvcc compiler:

In [ ]:
!nvcc vadd_target.c -o vadd_target -Xcompiler -fopenmp

And now executing:


In [ ]:
!export OMP_TARGET_OFFLOAD=mandatory
!./vadd_target

 vectors added with 0 errors
Init time:    0.006s
Compute time: 0.000s
Test time:    0.000s
Total time:   0.007s


In [5]:
%%writefile experiment_with_function.cc
#include <omp.h>
#include <stdio.h>

const int N = 100000;

int add_one(int x)
{
    return x + 1;
}

int sum_array(int *array) {
  int sum = 0;
  for(int i = 0;i < N;i++) {
    sum = sum + array[i];
  }
  return sum;
}

int main()
{
    int array[N];

    for(int i = 0;i < N;i++)
    {
        array[i] = i;
    }
    int result = sum_array(array);
    printf("Sum before incrementing: %d\n",result);

   #pragma omp target
   #pragma omp loop
   for (int i=0; i<N; i++){
      array[i] = add_one(array[i]);
   }

   result = sum_array(array);
   printf("Sum after incrementing: %d\n",result);


}



Overwriting experiment_with_function.cc


In [6]:
!!nvcc experiment_with_function.cc -o experiment_with_function -Xcompiler -fopenmp

[]

In [8]:
!export OMP_TARGET_OFFLOAD=mandatory
!time ./experiment_with_function

Sum before incrementing: 704982704
Sum after incrementing: 705082704

real	0m0.004s
user	0m0.001s
sys	0m0.002s


In [9]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [10]:
!gcc --version

gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

